> 安装依赖

In [1]:
%pip install ultralytics psutil requests pyyaml zipfile36 tqdm
%pip install wget pynvml GPUtil

Looking in indexes: https://mirrors.tencent.com/pypi/simple/

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.tencent.com/pypi/simple/

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


> 主路径

In [2]:
from pathlib import Path
Path.cwd()

PosixPath('/')

> 显卡信息

In [2]:
import torch

def get_gpu_info_torch():
    print(f"--- GPU Info (PyTorch) ---")
    if torch.cuda.is_available():
        device_count = torch.cuda.device_count()
        print(f"CUDA 可用，设备数量: {device_count}")
        for i in range(device_count):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            # 注意: torch 默认不直接提供实时的显存占用数值(used memory)，
            # 通常需要结合 torch.cuda.memory_allocated() 查看当前进程占用的显存
    else:
        print("未检测到 CUDA 环境")

get_gpu_info_torch()

/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


--- GPU Info (PyTorch) ---
CUDA 可用，设备数量: 1
GPU 0: NVIDIA A10


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
!nvidia-smi

Failed to initialize NVML: Driver/library version mismatch
NVML library version: 580.95


> 下载数据集

In [ ]:
import os
import requests
import zipfile
import shutil
from tqdm import tqdm

# === 设置下载链接 ===
DATASET_URL = "https://github.com/Norsico/yolodata/releases/download/0.0.0/SODA10M_82_3k.zip"
ZIP_FILE = "/workspace/SODA10M_82_3k.zip"
DATASET_DIR = "/workspace/datasets/SODA10M_82_3k"  # 期望的最终根目录

def download_file(url, filename):
    """使用 requests + tqdm 实现带进度条的下载"""
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024 # 1 Kibibyte
    
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True, desc="正在下载")
    
    with open(filename, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("⚠️ 错误：下载可能不完整")
    else:
        print("✅ 下载完成")

def fix_nested_dir(target_dir):
    """
    检查是否存在双层嵌套 (例如 target_dir/target_dir/images)，
    如果存在，将内部文件移动到 target_dir 并删除多余层级。
    """
    # 获取 target_dir 的文件夹名称，例如 "SODA10M_82_3k"
    folder_name = os.path.basename(target_dir)
    # 猜测的嵌套路径，例如 ".../SODA10M_82_3k/SODA10M_82_3k"
    nested_path = os.path.join(target_dir, folder_name)
    
    if os.path.exists(nested_path) and os.path.isdir(nested_path):
        print(f"⚠️ 检测到多层嵌套: {nested_path}")
        print("🔧 正在修正目录结构...")
        
        # 移动所有文件到上一层
        for item in os.listdir(nested_path):
            src = os.path.join(nested_path, item)
            dst = os.path.join(target_dir, item)
            shutil.move(src, dst)
            
        # 删除空的嵌套文件夹
        os.rmdir(nested_path)
        print("✅ 目录结构修正完成！")
    else:
        print("✅ 目录结构正常，无需修正。")

# ================= 主流程 =================

# 1. 下载逻辑
if not os.path.exists(ZIP_FILE):
    # 如果目标文件夹已经存在，说明可能之前解压过
    if not os.path.exists(DATASET_DIR):
        print(f"⬇️ 开始下载数据集: {DATASET_URL}")
        download_file(DATASET_URL, ZIP_FILE)
    else:
        print(f"✅ 数据集目录 {DATASET_DIR} 已存在，无需下载压缩包")
else:
    print(f"✅ 数据集压缩包 {ZIP_FILE} 已存在，跳过下载")

# 2. 解压逻辑
if not os.path.exists(DATASET_DIR):
    if os.path.exists(ZIP_FILE):
        print(f"📦 正在解压数据集到 {DATASET_DIR} ...")
        os.makedirs(DATASET_DIR, exist_ok=True)
        
        with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
            for member in tqdm(zip_ref.infolist(), desc="正在解压"):
                zip_ref.extract(member, DATASET_DIR)
        
        print(f"✅ 解压完成")

        # === 3. 新增：修正嵌套目录 ===
        fix_nested_dir(DATASET_DIR)

        # === 4. 删除压缩包 ===
        print(f"🗑️ 正在删除压缩包以释放空间: {ZIP_FILE}")
        os.remove(ZIP_FILE)
        print("✅ 压缩包已删除")
    else:
        print("⚠️ 错误：未找到压缩包，无法解压")
else:
    print(f"✅ 数据集目录 {DATASET_DIR} 已存在，跳过解压")
    # 再次检查目录结构（防止上次解压了但没修正）
    fix_nested_dir(DATASET_DIR)
    
    if os.path.exists(ZIP_FILE):
        os.remove(ZIP_FILE)
        print(f"🗑️ 检测到残留的压缩包 {ZIP_FILE}，已删除")

⬇️ 开始下载数据集: https://github.com/Norsico/yolodata/releases/download/0.0.0/SODA10M_82_3k.zip


正在下载: 100%|██████████| 945M/945M [00:44<00:00, 21.1MiB/s] 


✅ 下载完成
📦 正在解压数据集到 /workspace/datasets/SODA10M_82_3k ...


正在解压: 100%|██████████| 6007/6007 [00:03<00:00, 1750.92it/s]


✅ 解压完成
⚠️ 检测到多层嵌套: /workspace/datasets/SODA10M_82_3k/SODA10M_82_3k
🔧 正在修正目录结构...
✅ 目录结构修正完成！
🗑️ 正在删除压缩包以释放空间: /workspace/SODA10M_82_3k.zip
✅ 压缩包已删除


In [ ]:
import os
from ultralytics import YOLO

# ================= 配置区域 =================
# 1. 模型设置
CUSTOM_CFG_NAME = '/workspace/yolo11n.yaml'

if os.path.exists(CUSTOM_CFG_NAME):
    MODEL_CFG = CUSTOM_CFG_NAME
    print(f"✅ 找到自定义模型配置: {MODEL_CFG}")
else:
    print(f"⚠️ 未找到 {CUSTOM_CFG_NAME}，将使用标准 yolo11n.yaml 进行演示")
    MODEL_CFG = 'yolo11n.yaml' 

# 2. 数据集设置
DATA_PATH = '/workspace/datasets/SODA10M_82_3k/soda10m.yaml' 

# 3. 训练参数
EPOCHS = 100            
BATCH_SIZE = 48         # 云端显存通常较大(如T4/P100)，可以尝试 16 或 32
IMG_SIZE = 960          
DEVICE = '0'            
WORKERS = 16             # 调用的CPU核心数

# 4. 保存路径设置
PROJECT_DIR = '/workspace/runs/train'  
EXP_NAME = 'scratch_yolo11n_soda10m_3k_960' 
# =======================================================

def train_model():
    # 构造上次训练的断点路径
    last_ckpt_path = os.path.join(PROJECT_DIR, EXP_NAME, 'weights', 'last.pt')
    
    # === 自动断点续训逻辑 ===
    resume_training = False
    
    if os.path.exists(last_ckpt_path):
        print(f"✅ 检测到上次未完成的训练，正在从断点恢复: {last_ckpt_path}")
        # 加载断点模型
        model = YOLO(last_ckpt_path)
        resume_training = True
    else:
        print(f"🆕 未检测到断点，开始新的训练: {MODEL_CFG}")
        # 加载新模型 (Build from YAML for scratch training)
        model = YOLO(MODEL_CFG)
        
        # 如果你想加载预训练权重来微调，取消下面这行的注释:
        # model = YOLO('yolo11n.pt') 
        
        resume_training = False

    # === 开始训练 ===
    results = model.train(
        data=DATA_PATH,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        device=DEVICE,
        workers=WORKERS,
        project=PROJECT_DIR,
        name=EXP_NAME,
        resume=resume_training, # 关键参数：是否续训
        exist_ok=True,          # 允许覆盖同名文件夹(配合resume使用)
        cache=True,             # 缓存图片到内存，加速训练
        amp=True                # 开启混合精度训练(通常默认开启，云端建议开启)
    )
    
    print(f"\n🎉 训练结束！最佳模型保存在: {os.path.join(PROJECT_DIR, EXP_NAME, 'weights', 'best.pt')}")
    return model

trained_model = train_model()


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


⚠️ 未找到 /workspace/yolo11n.yaml，将使用标准 yolo11n.yaml 进行演示
🆕 未检测到断点，开始新的训练: yolo11n.yaml
Ultralytics 8.3.235 🚀 Python-3.11.1 torch-2.8.0+cu128 CUDA:0 (NVIDIA A10, 22588MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=48, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/workspace/datasets/SODA10M_82_3k/soda10m.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=scratch_yolo11n_soda10m_3k_960, nbs=

/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


  8                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1    249728  ultralytics.nn.modules.block.C2PSA           [256, 256, 1]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  1    111296  ultralytics.nn.modules.block.C3k2            [384, 128, 1, False]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1     32096  ultralytics.nn.modules.block.C3k2            [256,

In [ ]:
import shutil
import os

# 配置路径
source_dir = '/workspace/runs/train/scratch_yolo11n_soda10m_3k_960'  # 你的训练结果目录
output_filename = '/workspace/scratch_yolo11n_soda10m_3k_960'                    # 输出文件名 (不需要加 .zip，会自动加)

# 检查源目录是否存在
if os.path.exists(source_dir):
    print(f"正在打包文件夹: {source_dir} ...")
    
    # 创建压缩包 ('zip' 格式)
    shutil.make_archive(output_filename, 'zip', source_dir)
    
    print(f"✅ 打包成功！文件已保存为: {output_filename}.zip")
else:
    print(f"❌ 错误: 找不到目录 {source_dir}")

正在打包文件夹: /workspace/runs/train/scratch_yolo11n_soda10m_3k ...
✅ 打包成功！文件已保存为: /workspace/scratch_yolo11n_soda10m_3k.zip
